In [ ]:
import numpy as np
import pandas as pd

In [ ]:
class BollingerBands:
    def __init__(self, prices, window=20, num_std_dev=2):
        self.prices = prices
        self.window = window
        self.num_std_dev = num_std_dev
        self.middle_band, self.upper_band, self.lower_band = self.calculate_bands()
    
    def calculate_bands(self):
        middle_band = self.prices.rolling(self.window).mean()
        std_dev = self.prices.rolling(self.window).std()
        upper_band = middle_band + (std_dev * self.num_std_dev)
        lower_band = middle_band - (std_dev * self.num_std_dev)
        return middle_band, upper_band, lower_band

    def cross_signal(self):
        buy_signal = (self.prices > self.lower_band).shift(1) & (self.prices <= self.lower_band)
        sell_signal = (self.prices < self.lower_band) | (self.prices > self.upper_band)
        return buy_signal, sell_signal

    def aux_signal(self):
        buy_signal = self.prices < self.lower_band
        sell_signal = self.prices > self.upper_band
        return buy_signal, sell_signal


In [ ]:
class EMA:
    def __init__(self, prices, short_window=5, long_window=10):
        self.prices = prices
        self.short_window = short_window
        self.long_window = long_window
        self.short_ema = self.calculate_ema(short_window)
        self.long_ema = self.calculate_ema(long_window)
    
    def calculate_ema(self, window):
        return self.prices.ewm(span=window, adjust=False).mean()

    def cross_signal(self):
        buy_signal = (self.short_ema > self.long_ema) & (self.short_ema.shift(1) <= self.long_ema.shift(1))
        sell_signal = (self.short_ema < self.long_ema) & (self.short_ema.shift(1) >= self.long_ema.shift(1))
        return buy_signal, sell_signal

    def aux_signal(self):
        buy_signal = self.short_ema > self.long_ema
        sell_signal = self.short_ema < self.long_ema
        return buy_signal, sell_signal


In [ ]:
class KeltnerChannel:
    def __init__(self, prices, window=20, multiplier=2):
        self.prices = prices
        self.window = window
        self.multiplier = multiplier
        self.middle_band, self.upper_band, self.lower_band = self.calculate_bands()
    
    def calculate_bands(self):
        middle_band = self.prices.ewm(span=self.window, adjust=False).mean()
        atr = self.prices.rolling(window=self.window).apply(lambda x: np.max(x) - np.min(x), raw=True)
        upper_band = middle_band + (atr * self.multiplier)
        lower_band = middle_band - (atr * self.multiplier)
        return middle_band, upper_band, lower_band

    def cross_signal(self):
        buy_signal = (self.prices > self.lower_band).shift(1) & (self.prices <= self.lower_band)
        sell_signal = (self.prices < self.lower_band) | (self.prices > self.upper_band)
        return buy_signal, sell_signal

    def aux_signal(self):
        buy_signal = self.prices < self.lower_band
        sell_signal = self.prices > self.upper_band
        return buy_signal, sell_signal


In [ ]:
class MACD:
    def __init__(self, prices, short_window=12, long_window=26, signal_window=9):
        self.prices = prices
        self.short_window = short_window
        self.long_window = long_window
        self.signal_window = signal_window
        self.macd_line, self.signal_line, self.histogram = self.calculate_macd()
    
    def calculate_macd(self):
        short_ema = self.prices.ewm(span=self.short_window, adjust=False).mean()
        long_ema = self.prices.ewm(span=self.long_window, adjust=False).mean()
        macd_line = short_ema - long_ema
        signal_line = macd_line.ewm(span=self.signal_window, adjust=False).mean()
        histogram = macd_line - signal_line
        return macd_line, signal_line, histogram

    def cross_signal(self):
        buy_signal = (self.macd_line > self.signal_line) & (self.macd_line.shift(1) <= self.signal_line.shift(1))
        sell_signal = (self.macd_line < self.signal_line) & (self.macd_line.shift(1) >= self.signal_line.shift(1))
        return buy_signal, sell_signal

    def aux_signal(self):
        buy_signal = self.macd_line > self.signal_line
        sell_signal = self.macd_line < self.signal_line
        return buy_signal, sell_signal


In [ ]:
class VortexIndicator:
    def __init__(self, prices, window=14):
        self.prices = prices
        self.window = window
        self.vi_plus, self.vi_minus = self.calculate_vi()
    
    def calculate_vi(self):
        high_low = self.prices['High'] - self.prices['Low']
        high_close_prev = abs(self.prices['High'] - self.prices['Close'].shift(1))
        low_close_prev = abs(self.prices['Low'] - self.prices['Close'].shift(1))

        true_range = pd.DataFrame([high_low, high_close_prev, low_close_prev]).max()

        vm_plus = abs(self.prices['High'] - self.prices['Low'].shift(1))
        vm_minus = abs(self.prices['Low'] - self.prices['High'].shift(1))

        sum_tr = true_range.rolling(window=self.window).sum()
        sum_vm_plus = vm_plus.rolling(window=self.window).sum()
        sum_vm_minus = vm_minus.rolling(window=self.window).sum()

        vi_plus = sum_vm_plus / sum_tr
        vi_minus = sum_vm_minus / sum_tr

        return vi_plus, vi_minus

    def cross_signal(self):
        buy_signal = (self.vi_plus > self.vi_minus) & (self.vi_plus.shift(1) <= self.vi_minus.shift(1))
        sell_signal = (self.vi_plus < self.vi_minus) & (self.vi_plus.shift(1) >= self.vi_minus.shift(1))
        return buy_signal, sell_signal

    def aux_signal(self):
        buy_signal = self.vi_plus > self.vi_minus
        sell_signal = self.vi_plus < self.vi_minus
        return buy_signal, sell_signal


In [ ]:
class ADX:
    def __init__(self, prices, window=14):
        self.prices = prices
        self.window = window
        self.adx = self.calculate_adx()
    
    def calculate_adx(self):
        plus_dm = self.prices.diff().clip(lower=0)
        minus_dm = -self.prices.diff().clip(upper=0)
        atr = self.prices.rolling(window=self.window).apply(lambda x: np.max(x) - np.min(x), raw=True)
        plus_di = 100 * (plus_dm / atr).ewm(span=self.window, adjust=False).mean()
        minus_di = 100 * (minus_dm / atr).ewm(span=self.window, adjust=False).mean()
        dx = 100 * np.abs((plus_di - minus_di) / (plus_di + minus_di))
        adx = dx.ewm(span=self.window, adjust=False).mean()
        return adx

    def aux_signal(self):
        buy_signal = self.adx > 25
        sell_signal = self.adx < 25
        return buy_signal, sell_signal


In [ ]:
class RSI:
    def __init__(self, prices, window=14):
        self.prices = prices
        self.window = window
        self.rsi = self.calculate_rsi()
    
    def calculate_rsi(self):
        delta = self.prices.diff(1)
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        avg_gain = gain.rolling(window=self.window).mean()
        avg_loss = loss.rolling(window=self.window).mean()
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
        return rsi

    def aux_signal(self):
        buy_signal = self.rsi < 30
        sell_signal = self.rsi > 70
        return buy_signal, sell_signal


In [ ]:
class CombinedIndicatorStrategy:
    def __init__(self, indicators):
        self.indicators = indicators
        self.validate_indicators()
    
    def validate_indicators(self):
        # Check the first indicator is a CROSS
        if not hasattr(self.indicators[0], 'cross_signal'):
            raise ValueError("The first indicator must be a CROSS type.")

        # Check the second and third indicators, if present, are AUX
        if len(self.indicators) > 1:
            if not hasattr(self.indicators[1], 'aux_signal'):
                raise ValueError("The second indicator must be an AUX type.")
        if len(self.indicators) > 2:
            if not hasattr(self.indicators[2], 'aux_signal'):
                raise ValueError("The third indicator must be an AUX type.")
    
    def generate_signals(self):
        # Initialize lists to hold buy and sell signals
        buy_signals = []
        sell_signals = []

        # Get the CROSS signals from the first indicator
        buy, sell = self.indicators[0].cross_signal()
        buy_signals.append(buy)
        sell_signals.append(sell)

        # Get AUX signals from the second indicator, if present
        if len(self.indicators) > 1:
            buy, sell = self.indicators[1].aux_signal()
            buy_signals.append(buy)
            sell_signals.append(sell)

        # Get AUX signals from the third indicator, if present
        if len(self.indicators) > 2:
            buy, sell = self.indicators[2].aux_signal()
            buy_signals.append(buy)
            sell_signals.append(sell)

        # Combine the signals
        combined_buy_signal = pd.concat(buy_signals, axis=1).all(axis=1)
        combined_sell_signal = pd.concat(sell_signals, axis=1).all(axis=1)

        return combined_buy_signal, combined_sell_signal


<!-- # Assuming df is a DataFrame with price data including columns 'High', 'Low', 'Close'
prices = df[['High', 'Low', 'Close']]

# Initialize indicators
vi = VortexIndicator(prices)
ema = EMA(prices, short_window=5, long_window=10)
rsi = RSI(prices)

# Combine indicators: CROSS (VI) and AUX (EMA, RSI)
strategy = CombinedIndicatorStrategy([vi, ema, rsi])
buy_signals, sell_signals = strategy.generate_signals() -->